In [11]:
import os
import cv2
import numpy as np

In [18]:
mask_dir = r'data2\val\masks'
labels_dir = r'data2\val\labels'
imgs_dir = r'data2\val\images'

In [ ]:
os.makedirs(labels_dir, exist_ok=True)

for mask_name, img_name  in zip(os.listdir(mask_dir), os.listdir(imgs_dir)):
    mask_path = os.path.join(mask_dir, mask_name)

    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    _, mask = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY)

    H, W = mask.shape

    contours, hierarchy = cv2.findContours(mask, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

    external_polygons = []
    internal_polygons = []

    for idx, cnt in enumerate(contours):
        if cv2.contourArea(cnt) > 200:
            polygon = []
            for point in cnt:
                x, y = point[0]
                polygon.append(x / W)
                polygon.append(y / H)
            if hierarchy[0][idx][3] == -1: 
                external_polygons.append(polygon)
            else:  
                internal_polygons.append(polygon)

    output_path = os.path.join(labels_dir, f"{img_name[:-4]}.txt")
    with open(output_path, 'w') as f:
        for polygon in external_polygons:
            f.write('0 ')
            f.write(' '.join(map(str, polygon)))
            f.write('\n')
        for polygon in internal_polygons:
            f.write('1 ')
            f.write(' '.join(map(str, polygon)))
            f.write('\n')


In [50]:

# works fine, not used
def visualize_polygons(image_shape, txt_file):
    H, W = image_shape
    image = np.zeros((H, W, 3), dtype=np.uint8)
    
    with open(txt_file, 'r') as f:
        lines = f.readlines()
    
    for line in lines:
        if line.strip():
            points = list(map(float, line.strip().split()[1:]))
            polygon = [(int(points[i] * W), int(points[i+1] * H)) for i in range(0, len(points), 2)]
            
            print(line.strip().split()[0])
            if line.strip().split()[0] == '0':
                color = (0, 255, 0)
            else:
                color = (255, 0, 0)
            cv2.polylines(image, [np.array(polygon, dtype=np.int32)], isClosed=True, color=color, thickness=2)
    
    cv2.imshow('polygon', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
# visualize_polygons((512, 512), r'data\train\labels\15085_1_mask.txt')